In [15]:
import napari
from pathlib import Path
from skimage import io
import os
import czifile
import math
import numpy as np
from tqdm.auto import tqdm

### Zeiss lattice lightsheet

In [3]:
os.listdir(Path(r'Y:\The_Holy_de_Broglies\2025-08-13 Lattice Lightsheet\Macrophages\2025-08-13\mouse'))

['middle-01.czi',
 'middle-01_MIP.czi',
 'middle-01_processed.czi',
 'middle-01_processed.ome.tiff',
 'middle-02.czi',
 'middle-02_DCVSettingsTest.czi',
 'middle-02_DCVSettingsTest.czt',
 'middle-02_MIP.czi',
 'middle-02_processed.czi',
 'middle-03.czi',
 'middle-03_DCVSettingsTest.czi',
 'middle-03_DCVSettingsTest.czt',
 'middle-03_MIP.czi',
 'middle-03_processed.czi',
 'middle-04.czi',
 'middle-04_MIP.czi',
 'middle-04_processed.czi',
 'middle-05.czi',
 'middle-05_MIP.czi',
 'middle-05_processed.czi',
 'middle-06.czi',
 'middle-06_MIP.czi',
 'middle-06_processed.czi',
 'middle-11-Lattice Lightsheet-01.czi',
 'middle-11.czi',
 'middle-11_MIP.czi',
 'middle-15-Lattice Lightsheet-02.czi',
 'middle-15.czi',
 'middle-15_MIP.czi',
 'wildtype-01-deskewed-deconv.czi',
 'wildtype-01-deskewed.czi',
 'wildtype-01.czi',
 'wildtype-01.ome.tiff',
 'wildtype-01_MIP.czi',
 'wildtype-01_MIP.ome.tiff',
 'wildtype-01_processed-01.czi',
 'wildtype-01_processed.czi']

In [2]:
viewer = napari.Viewer(title = 'zeiss')

### Mouse macrophages

In [4]:
image_path = Path(r'Z:\The_Holy_de_Broglies\2025-08-13 Lattice Lightsheet\Macrophages\2025-08-13\mouse\wildtype-01-deskewed-deconv.czi')

In [5]:
%%time 

image = czifile.imread(image_path)

CPU times: total: 46 s
Wall time: 2min 37s


In [6]:
image.shape

(1, 2, 114, 7576, 6407, 1)

In [9]:
image_reshaped = image[0,...,0]
image_reshaped.shape

(2, 114, 7576, 6407)

### Deskew (no longer necessary)

In [49]:
from scipy.ndimage import shift
from tqdm.auto import tqdm
import numpy as np

In [56]:
C, Z, Y, X = image.shape

angle_deg = 31.8
xy_um = 0.145
z_um  = 0.400
px_shift_per_z = (z_um / xy_um) * np.tan(np.deg2rad(angle_deg))

max_shift = int(np.ceil(abs((Z - 1) * px_shift_per_z)))
X_expanded = X + max_shift
deskewed = np.zeros((C, Z, Y, X_expanded), dtype=image.dtype)

x0 = max_shift // 2
sign = +1.0  # positive sign as requested

for c in range(C):
    for z in tqdm(range(Z), leave=False, desc=f"Deskew C{c}"):
        x_pos = x0 + sign * z * px_shift_per_z
        x_int = int(np.floor(x_pos))
        frac  = x_pos - x_int

        x_start = max(0, min(X_expanded - X, x_int))
        canvas = np.zeros((Y, X_expanded), dtype=image.dtype)
        canvas[:, x_start : x_start + X] = image[c, z]

        if abs(frac) > 1e-6:
            canvas = shift(canvas, shift=(0, frac), order=1, mode="nearest", prefilter=False)

        deskewed[c, z] = canvas

Deskew C0:   0%|          | 0/2251 [00:00<?, ?it/s]

Deskew C1:   0%|          | 0/2251 [00:00<?, ?it/s]

## Visualise

In [8]:
viewer.add_image(image, channel_axis=1, name = 'wildtype deskewed')

[<Image layer 'wildtype deskewed' at 0x1def55a4e50>,
 <Image layer 'wildtype deskewed [1]' at 0x1deb9f39fd0>]

In [10]:
viewer.add_image(image_reshaped, channel_axis=0, name = 'wildtype deskewed reshaped')

[<Image layer 'wildtype deskewed reshaped' at 0x1de861225d0>,
 <Image layer 'wildtype deskewed reshaped [1]' at 0x1def787f150>]

In [12]:
viewer.layers

[<Image layer 'wildtype deskewed reshaped' at 0x1de861225d0>, <Image layer 'wildtype deskewed reshaped [1]' at 0x1def787f150>]

In [14]:
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'μm'

In [15]:
for i in [0,1]:
    print(viewer.layers[i].contrast_limits)

[154.66696914700543, 857.9282084521649]
[300.0, 800.0]
Rendering frames...


100%|██████████████████████████████████████████████████████| 91/91 [00:04<00:00, 19.08it/s]


Rendering frames...


  2%|█▏                                                     | 2/91 [00:00<00:07, 11.97it/s]


In [16]:
print()


Rendering frames...


  0%|                                                      | 1/601 [00:00<01:30,  6.65it/s]


In [17]:
print()


Rendering frames...


100%|████████████████████████████████████████████████████| 601/601 [00:29<00:00, 20.60it/s]


Rendering frames...


100%|████████████████████████████████████████████████████| 601/601 [00:28<00:00, 20.87it/s]


In [13]:
for i in [0,1]:
    viewer.layers[i].scale = [0.4,0.145,0.145]

### Scape

In [5]:
os.listdir(Path(r'Y:\The_Holy_de_Broglies\ASI SCAPE\20250813\Processed'))

['20250813_macrophages_GLA3488_multiPos_lowerLaserP_MMStack_Pos0.ome.deskewed_TRUE_Pos00_CH00_T000.tif',
 'MAX20250813_macrophages_GLA3488_multiPos_lowerLaserP_MMStack_Pos0.ome_deskewed_TRUE_Pos00_CH00_T000.tif',
 'Thumbs.db']

In [6]:
viewer = napari.Viewer(title = 'scape')

### Mouse macrophages

In [7]:
image_path = Path(r'Y:\The_Holy_de_Broglies\ASI SCAPE\20250813\Processed\20250813_macrophages_GLA3488_multiPos_lowerLaserP_MMStack_Pos0.ome.deskewed_TRUE_Pos00_CH00_T000.tif')

In [9]:
%%time 

image = io.imread(image_path)

CPU times: total: 7.48 s
Wall time: 25.5 s


In [10]:
image.shape

(3, 81, 15, 1321, 1005)

### Separate positions and tile into pseudo config

In [16]:
# Input: image shaped (C, Z, P, Y, X) e.g. (3, 81, 15, 1321, 1005)
C, Z, P, Y, X = image.shape

# Choose a compact grid to tile the P=15 positions (row-major, arbitrary order)
cols = math.ceil(math.sqrt(P))
rows = math.ceil(P / cols)

# Allocate output mosaic: (C, Z, Y_total, X_total)
mosaic = np.zeros((C, Z, rows * Y, cols * X), dtype=image.dtype)

# Tile positions into the mosaic
p = 0
for r in tqdm(range(rows)):
    for c in tqdm(range(cols)):
        if p >= P:
            break
        ys, ye = r * Y, (r + 1) * Y
        xs, xe = c * X, (c + 1) * X
        mosaic[:, :, ys:ye, xs:xe] = image[:, :, p, :, :]
        p += 1

# Result: 'mosaic' is (C, Z, Y_total, X_total)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [49]:
from scipy.ndimage import shift
from tqdm.auto import tqdm
import numpy as np

In [56]:
C, Z, Y, X = image.shape

angle_deg = 31.8
xy_um = 0.145
z_um  = 0.400
px_shift_per_z = (z_um / xy_um) * np.tan(np.deg2rad(angle_deg))

max_shift = int(np.ceil(abs((Z - 1) * px_shift_per_z)))
X_expanded = X + max_shift
deskewed = np.zeros((C, Z, Y, X_expanded), dtype=image.dtype)

x0 = max_shift // 2
sign = +1.0  # positive sign as requested

for c in range(C):
    for z in tqdm(range(Z), leave=False, desc=f"Deskew C{c}"):
        x_pos = x0 + sign * z * px_shift_per_z
        x_int = int(np.floor(x_pos))
        frac  = x_pos - x_int

        x_start = max(0, min(X_expanded - X, x_int))
        canvas = np.zeros((Y, X_expanded), dtype=image.dtype)
        canvas[:, x_start : x_start + X] = image[c, z]

        if abs(frac) > 1e-6:
            canvas = shift(canvas, shift=(0, frac), order=1, mode="nearest", prefilter=False)

        deskewed[c, z] = canvas

Deskew C0:   0%|          | 0/2251 [00:00<?, ?it/s]

Deskew C1:   0%|          | 0/2251 [00:00<?, ?it/s]

In [17]:
mosaic.shape

(3, 81, 5284, 4020)

In [23]:
mosaic = np.max(mosaic, axis=0)

## Visualise

In [24]:
viewer.add_image(mosaic, name = 'mosaic')

<Image layer 'mosaic' at 0x1be06a59b90>

In [25]:
viewer.layers

[<Image layer 'mosaic' at 0x1be06a59b90>]

In [27]:
for i in viewer.layers:
    i.scale = [0.5, 0.391, 0.391]

In [28]:
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'μm'

In [31]:
viewer.scale_bar.font_size = 20
viewer.scale_bar.ticks = False

Rendering frames...


100%|████████████████████████████████████████| 151/151 [00:05<00:00, 26.56it/s]


Rendering frames...


100%|████████████████████████████████████████| 151/151 [00:05<00:00, 26.98it/s]


In [29]:
for i in [0,1]:
    print(viewer.layers[i].contrast_limits)

[0.0, 117.0]


IndexError: list index out of range